In [142]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd

# Задание №1

In [143]:
URL = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг']

In [173]:
habr_post = pd.DataFrame()
res = requests.get(URL)
soup = BeautifulSoup(res.text)
last_pages = int(''.join(re.findall(r'\d+', soup.find('a',class_='toggle-menu__item-link toggle-menu__item-link_pagination toggle-menu__item-link_bordered').get('href'))))
for i in range(1,last_pages+1):
    time.sleep(0.2)
    page ='https://habr.com/ru/all/page' + str(i) + '/'
    rest_page = requests.get(page)
    soup_page = BeautifulSoup(rest_page.text)
    # Все новости
    posts = soup_page.find_all('article', class_='post')
    for post in posts:
        post_id = post.parent.attrs.get('id')
        if not post_id:
            continue
        else:
            if post.find('div', class_='post__text') is not None:
                preview_new = post.find('div', class_='post__text')
                if any([desired in preview_new.text.lower() for desired in KEYWORDS]):
                    link = post.find('a', class_='post__title_link').attrs.get('href')
                    rest_post = requests.get(link)
                    soup_post = BeautifulSoup(rest_post.text)
                    date = pd.to_datetime(soup_post.find('span', class_='post__time').attrs.get('data-time_published'), dayfirst=True).date()
                    title = soup_post.find('h1', class_='post__title post__title_full').text.strip()
                    text = soup_post.find('div', class_='post__body post__body_full').text.strip()
                    row = {'date': date, 'title': title, 'link': link, 'text': text}
                    habr_post = pd.concat([habr_post, pd.DataFrame([row])])
                    break
                else:
                    link = post.find('a', class_='post__title_link').attrs.get('href')
                    full_post = requests.get(link)
                    soup_full_post = BeautifulSoup(full_post.text)
                    full_text = soup_full_post.find('div', class_='post__body post__body_full')
                    if any([desired in full_text.text.lower() for desired in KEYWORDS]):
                        link = post.find('a', class_='post__title_link').attrs.get('href')
                        date = pd.to_datetime(soup_full_post.find('span', class_='post__time').attrs.get('data-time_published'), dayfirst=True).date()
                        title = soup_full_post.find('h1', class_='post__title post__title_full').text.strip()
                        text = soup_full_post.find('div', class_='post__body post__body_full').text.strip()
                        row = {'date': date, 'title': title, 'link': link, 'text': text}
                        habr_post = pd.concat([habr_post, pd.DataFrame([row])])
                        break
    

In [174]:
habr_post.head()

,date,title,link,text
0,2021-04-23,"Мы выяснили, что люди копируют со Stack Overfl...",https://habr.com/ru/company/productivity_insid...,"Говорят, что в каждой шутке есть доля правды. ..."
0,2021-04-22,Как мы сетапили монорепозиторий с SSR и SPA дл...,https://habr.com/ru/company/kts/blog/553806/,"Привет, хабр! Меня зовут Фёдор и я фронтенд-ра..."
0,2021-04-22,«Кадровый голод» или почему IT-компании находя...,https://habr.com/ru/company/automacon/blog/553...,Последние годы в IT-отрасли наблюдается очевид...
0,2021-04-21,Временные ряды. Простые решения,https://habr.com/ru/post/553658/,"Привет, Хабр!\n\r\nВ этой статье мы рассмотрим..."
0,2021-04-21,От студента до учителя: как разобраться в веб-...,https://habr.com/ru/company/geekbrains/blog/55...,"Хоть кому-то и может показаться, что веб-разра..."


# Задание №2

In [249]:
emails = pd.DataFrame()
EMAILs = ['lp43@inbox.ru', 'valdis171186@gmail.com', 'shen22@rambler.ru']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
#params = {
#    'emailAddresses': ['shen22@rambler.ru'],
#}
headers = {
     'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.152 Mobile Safari/537.36'
    ,'Vaar-Version': '0' 
    , 'Vaar-Header-App-Product': 'hackcheck-web-avast'
    , 'Vaar-Header-App-Product-Name': 'hackcheck-web-avast'
    , 'Vaar-Header-App-Build-Version': '1.0.0'
}

for email in EMAILs:
    params['emailAddresses']= email.split()
    req = requests.post(URL, json=params, headers=headers)
    time.sleep(3)
    if len(req.json()['breaches']) == 0:
        date = 'нет данных'
        sourse = 'нет данных'
        description = 'нет данных'
        row = {'email': email, 'date': date, 'sourse': sourse, 'description': description}
        emails = pd.concat([emails, pd.DataFrame([row])])
    else:
        for i in req.json()['breaches'].keys():
            date = pd.to_datetime(req.json()['breaches'][i]['publishDate']).date()
            sourse = req.json()['breaches'][i]['site']
            description = req.json()['breaches'][i]['description']
            row = {'email': email, 'date': date, 'sourse': sourse, 'description': description}
            emails = pd.concat([emails, pd.DataFrame([row])])
emails

,email,date,sourse,description
0,lp43@inbox.ru,2020-05-21,vk.com,"At some time in 2020, the Russian social netwo..."
0,lp43@inbox.ru,2017-06-14,nnm-club.me,"In September 2013, Russian torrent tracker NNM..."
0,lp43@inbox.ru,2018-01-20,k-vrachu.ru,"In January 2016, K-Vrachu's database was alleg..."
0,valdis171186@gmail.com,нет данных,нет данных,нет данных
0,shen22@rambler.ru,2020-05-21,vk.com,"At some time in 2020, the Russian social netwo..."
0,shen22@rambler.ru,2016-10-21,myspace.com,"Shortly before the 2016 Memorial Day weekend, ..."
0,shen22@rambler.ru,2016-11-01,qip.ru,"In 2011, Russian instant messaging service pro..."
0,shen22@rambler.ru,2016-10-24,dropbox.com,Cloud storage company Dropbox suffered a major...
0,shen22@rambler.ru,2016-10-25,last.fm,"In March 2012, the British music streaming ser..."
0,shen22@rambler.ru,2020-12-03,worldofauto.ru,"In November 2020, a collection of over 23,000 ..."
